# Find duplicate candidates


Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and is used to find potential duplicated catchments within the dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas
* shapely
* textdistance
* tqdm
* warnings

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* data/streamflow/estreams_gauging_stations.csv

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References


## Observations
* As this step is rather qualitative, we believe that the users can also adapt the conditons accordinly. 


# Import modules

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import tqdm as tqdm
import os
import warnings
import textdistance
from shapely.geometry import Point

# Configurations

In [ ]:
# Only editable variables
# Relative path to your local directory
PATH = "../../.."

# Suppress all warnings
warnings.filterwarnings("ignore")

# Constrains
JARO_THRESHOLD = 0.75
SPATIAL_THRESHOLD = 1000
PROVIDER_THRESHOLD = 0.9

* #### The users should NOT change anything in the code below here.


In [ ]:
# Non-editable variables:
PATH_OUTPUT = "results/"

# Set the directory:
os.chdir(PATH)

# Import data
## Streamflow gauges network

In [ ]:
network_estreams = pd.read_csv('data/streamflow/estreams_gauging_stations.csv', encoding='utf-8')
network_estreams.set_index("basin_id", inplace = True)
network_estreams

# Processing

In [ ]:
# If we want to clip the data to be used:
df = network_estreams.iloc[:, :]

# Create a GeoDataFrame from DataFrame with WGS 84 coordinates
geometry = [Point(lon, lat) for lon, lat in zip(network_estreams['lon'], network_estreams['lat'])]
gdf_wgs84 = gpd.GeoDataFrame(network_estreams, geometry=geometry, crs='EPSG:4326')

# Reproject the GeoDataFrame to ETRS89 LAEA (EPSG:3035)
gdf_etrs89 = gdf_wgs84.to_crs(epsg=3035)

df = gdf_etrs89.loc[df.index, :]

In [ ]:
# Create a dictionary to store distances
distances = {}

# Calculate Jaro-Winkler distance for each unique pair of 'gauge_name'
for i, row1 in tqdm.tqdm(df.iterrows()):
    for j, row2 in gdf_etrs89.iterrows():
        # Skip self-comparisons
        if i != j:
            # Calculate gauge name distance
            try:
                gauge_distance = textdistance.jaro_winkler(row1['gauge_name'].lower(), row2['gauge_name'].lower())
            except: 
                gauge_distance = np.nan
                
            # Calculate river distance
            try:
                river_distance = textdistance.jaro_winkler(row1['river'].lower(), row2['river'].lower())
            except: 
                river_distance = np.nan
                

            provider_distance = row1['gauge_provider'].lower() == row2['gauge_provider'].lower()          
        
            # Calculate distance between points
            point1 = row1['geometry']
            point2 = row2['geometry']
            point_distance = point1.distance(point2)
            
            # gauges and river distance normalized:
            #dist_norm = (gauge_distance + river_distance)/2
            
            # Store distances along with first and second gauge indices only if gauge_distance > 0.9
            if (gauge_distance > JARO_THRESHOLD) & (river_distance > JARO_THRESHOLD) & (point_distance < SPATIAL_THRESHOLD) & (provider_distance == False):
                distances[(row1['gauge_name'], row2['gauge_name'])] = {'gauge_first_index': i, 
                                                                        'gauge_second_index': j,
                                                                        'gauge_distance': gauge_distance, 
                                                                        'river_distance': river_distance,
                                                                        'point_distance': point_distance,
                                                                        'provider_distance': provider_distance}
# Convert dictionary to DataFrame for visualization
dist_df = pd.DataFrame.from_dict(distances, orient='index')
dist_df.index.names = ['gauge_name1', 'gauge_name2']
dist_df.reset_index(inplace = True)
dist_df

In [ ]:
len(dist_df.gauge_first_index.unique())

In [ ]:
dist_df.head(50)

In [ ]:
most_common_name = dist_df['gauge_first_index'].value_counts().idxmax()

print("The most common name in the column is:", most_common_name)

In [ ]:
dist_df[dist_df.gauge_first_index == "ITLI0307"]

In [ ]:
dist_df['gauge_first_index'].value_counts().plot()

In [ ]:
# Here we add the list of duplicated suspects:

In [ ]:
network_estreams["duplicated_suspect"] = ""

In [ ]:
# First column:
for gauge in tqdm.tqdm(dist_df.gauge_first_index):
    
    duplicated_list = str(dist_df.gauge_second_index[dist_df.gauge_first_index == gauge].tolist()).replace("[", "")
    duplicated_list = duplicated_list.replace("]", "")
    duplicated_list = duplicated_list.replace("'", "")
    network_estreams.loc[gauge, "duplicated_suspect"] = duplicated_list
    network_estreams.loc[gauge, "duplicated_suspect"] = network_estreams.loc[gauge, "duplicated_suspect"]

In [ ]:
# Second column:
for gauge in tqdm.tqdm(dist_df.gauge_second_index):
    
    duplicated_list = str(dist_df.gauge_first_index[dist_df.gauge_second_index == gauge].tolist()).replace("[", "")
    duplicated_list = duplicated_list.replace("]", "")
    duplicated_list = duplicated_list.replace("'", "")
    network_estreams.loc[gauge, "duplicated_suspect"] = duplicated_list
    network_estreams.loc[gauge, "duplicated_suspect"] = network_estreams.loc[gauge, "duplicated_suspect"]

In [ ]:
network_estreams

In [ ]:
network_estreams.loc["ITLI0307", :]

In [ ]:
# Save the data:
network_estreams.to_csv("results/extras/estreams_gauging_stations_duplicates.csv", encoding='utf-8')

# End